In [ ]:
import pandas as pd
from google.colab import auth
import gspread as gp
from oauth2client.service_account import ServiceAccountCredentials
from google.auth import compute_engine
from google.colab import drive

In [ ]:
# Autenticação
auth.authenticate_user()
from google.auth import default
creds, _ = default()
gc = gp.authorize(creds)

In [ ]:
# Abrir planilha - EXTRACTION
sheet = ""
sh = gc.open(sheet)
worksheet = sh.get_worksheet(0)

In [ ]:
# TRANSFORMATION
df = pd.DataFrame(worksheet.get_all_records())
df.head()

In [ ]:
df_processado = df.copy()
df_processado[jogadores] = df_processado[jogadores].apply(filtrar_e_rastrear)

medias_finais = df_processado[jogadores].mean().sort_values(ascending=False).round(2)

df_votos_invalidos = pd.DataFrame(votos_fora_da_margem)

print("-"*30)
print("RANKING DE ESTRELAS (MÉDIA FILTRADA)")
print(medias_finais)
print("-"*30)

if not df_votos_invalidos.empty:
  print("\nRELAÇÃO DE VOTOS FORA DA MARGEM (Diferença > 2):")
  # Agrupamento por quem mais deu votos fora do padrão
  cornetas = df_votos_invalidos['Quem votou'].value_counts()
  print(df_votos_invalidos[['Quem votou', 'Em quem', 'Nota dada', 'Média do jogador']])
  print("\nRanking dos que mais votaram fora de consenso: ")
  print(cornetas)
else:
  print("\nNenhum voto fora da margem encontrado")

In [ ]:
# 1. Calcular a média original (antes do filtro)
medias_originais = df[jogadores].mean().round(2)

# 2. Calcular a média filtrada (usando o df_processado que criamos no passo anterior)
medias_filtradas = df_processado[jogadores].mean().round(2)

# 3. Criar uma tabela comparativa
df_comparativo = pd.DataFrame({
    'Média Original': medias_originais,
    'Média Filtrada': medias_filtradas
})

# 4. Calcular a diferença (Delta)
df_comparativo['Diferença'] = (df_comparativo['Média Filtrada'] - df_comparativo['Média Original']).round(2)

# Ordenar pelos que tiveram maior mudança (positiva ou negativa)
df_comparativo = df_comparativo.sort_values(by='Diferença', key=abs, ascending=False)

print("ANÁLISE DE IMPACTO DO FILTRO (Top 10 mudanças):")
print(df_comparativo.head(10))